# Building a Spam Filter with Naive Bayes
This project will classify messages as spam or non-spam using the Naive Bayes algorithm, and is based on a dataset of 5,572 SMS messages that are already classified. The dataset is put together by Tiago A. Almeida and José María Gómez Hidalgo.

Description|URL
-----------|---
Dataset URL|https://archive.ics.uci.edu/ml/datasets/sms+spam+collection
Dataset Download|https://dq-content.s3.amazonaws.com/433/SMSSpamCollection

---

Steps:
1. The computer learns how humans classify messages
2. Uses that human knowledge to estimate probabilities for new messages (probabilities for spam and non-spam)
3. Classifies a new message based on these probability values - if the probability for spam is greater, then it classifies the message as spam. Otherwise, it classifies it as non-spam (if the two probability values are equal, then we may need a human to classify the message).

In [2]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label','SMS'])

In [3]:
df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Exploring Dataset

In [4]:
df.shape

(5572, 2)

In [5]:
df['Label'].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

## Preparing Datasets
To test our spam filter once builts, we're going to need to test the filter and see how good it is with classifying new messages. For this, we're going to split our dataset into two categories:
1. Training set: which we'll use to "train" the computer how to classify messages (80% of the dataset, as we want to train the algorithm on as much data as possible, and have enough test data)
2. Test set: which we'll use to test how good a spam filter is with classifying new messages (20% of the dataset)

This means, the training set will have 4,458 messages, and the test set will have 1,114 messages.

The reason behind this is when the spam filter is ready, we're going to treat these messages as new and have the filter classify them. Then, compare the algorithm classification with that done by a human, and this way we'll see how good the spam filter really is.

Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80%, so we expect that more than 80% of the new messages will be classified correctly as spam or non-spam.

In [6]:
# Randomizing the entire dataset
df = df.sample(frac=1, random_state=1)

In [7]:
# Finding the split number
print(round(len(df) * 80 / 100))

4458


In [8]:
# Splitting the randomized dataset
training_df = df[:4458].reset_index(drop=True)
test_df = df[4458:].reset_index(drop=True)

In [9]:
training_df.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [10]:
# Checking training/test percentages from the main dataset
print(len(training_df) / len(df) * 100)
print(len(test_df) / len(df) * 100)

80.00717875089734
19.992821249102658


In [11]:
# Finding the percentages of spam and non-spam in the training dataset
training_df['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [12]:
# Finding the percentages of spam and non-spam in the test dataset
test_df['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The percentages of spam and non-spam labels are pretty similar to the original dataset during exploration.

## Data Cleaning
Before we jump into analysis, we need to do have a table with all the vocabulary as columns (vocabulary is a set of unique words), and their number of occurance in each message as rows.

Also, we'll remove the punctuation marks and lower the words cases for proper checking.

In [13]:
# Removing punctuation marks
training_df['SMS'] = training_df['SMS'].str.replace('\W',' ')

# Making it lowercase
training_df['SMS'] = training_df['SMS'].str.lower()

# Checking our work
# training_df.sample(10)

Now, we're going to create a list of unique words in all the messages

In [14]:
# Creating list of words for each message
training_df['SMS'] = training_df['SMS'].str.split()

# Making a unique list of vocabulary
vocab = []
for l in training_df['SMS']:
    for word in l:
        vocab.append(word)

vocab = list(set(vocab))
# Checking number of words we got
print(len(vocab))

7783


Our goal is to have the table with all vocabulary as columns and initial value of 0. We'll also include the label with it in a new DataFrame

In [15]:
# Creating main dictionary for all words counts
word_counts = {unique_word: [0] * len(training_df['SMS']) for unique_word in vocab}

In [16]:
# Counting words and updating dictionary
for index, sms in enumerate(training_df['SMS']):
    for word in sms:
        word_counts[word][index] += 1

In [17]:
# Converting dictionary to a DataFrame
detailed_training_df = pd.DataFrame(word_counts)

In [18]:
# Checking our work
detailed_training_df.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [19]:
# Concatenating the new DataFrame with the training set
# This way, we'll also have the columns 'Label' and 'SMS'
final_training_df = pd.concat([training_df, detailed_training_df], axis=1)

In [20]:
# Checking our work
final_training_df.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [21]:
# Validating a sample of our work
final_training_df.head().loc[:,['SMS','yep']]

,SMS,yep
0,"[yep, by, the, pretty, sculpture]",1
1,"[yes, princess, are, you, going, to, make, me,...",0
2,"[welp, apparently, he, retired]",0
3,[havent],0
4,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0


## Analysis

### Calculating Constants

We're done with data cleaning nad have a training set to work on. The Naive Bayes algorithm will need to work on require us to know the probability values of the two equations below to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

Also, to calculate  P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we use the below equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

So for this, we first need to calculate the below:
1. P(Spam) and P(Ham)
2. N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We have to know that:
1. N<sub>Spam</sub> is equal to the number of words in all the spam messages (not the number of spam messages, nor the total number of unique words in spam messages)
2. N<sub>Ham</sub> is equal to the number of words in all non-spam messages (not the number of non-spam messages, nor the total number of unique words in non-spam messages).

We'll also use Laplace smoothing and set α=1.

In [22]:
# Finding p_spam and p_ham
final_training_df['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [23]:
p_spam = 600 / len(final_training_df)
p_ham = 3858 / len(final_training_df)

In [24]:
# Finding n_spam and n_ham
n_spam = final_training_df[final_training_df['Label'] == 'spam'].iloc[:,2:].sum().sum()

In [25]:
# Quering n_ham using the same code as n_spam takes a lot of resource
# and causes the kernel to crash. It's better to use this approach
total_words = final_training_df.iloc[:,2:].sum().sum()
n_ham = total_words - n_spam

In [30]:
# Finding n_vocabulary
n_vocabulary = len(vocab)

# Assinging alpha variable
alpha = 1

In [31]:
# Printing the results
print('p_spam:', p_spam)
print('p_ham:', p_ham)
print('n_spam:', n_spam)
print('n_ham:', n_ham)
print('n_vocabulary:', n_vocabulary)
print('alpha:', alpha)

p_spam: 0.13458950201884254
p_ham: 0.8654104979811574
n_spam: 15190
n_ham: 57237
n_vocabulary: 7783
alpha: 1


### Calculating Parameters

Based on the above equations, P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) will vary depending on the individual words.

To calculate the above, we'll use the below equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [32]:
# Initializing two identical dictionaries for
# vocabularies to keep the parameters in
p_wi_spam_dict = {word: 0 for word in vocab}
p_wi_ham_dict = {word: 0 for word in vocab}

# Spliting the training data into spam and non-spam sets
training_spam = final_training_df[final_training_df['Label'] == 'spam']
training_ham = final_training_df[final_training_df['Label'] == 'ham']

In [33]:
training_ham['SMS'].apply(len).sum()

57237

In [34]:
# Looping through words and calculating
# parameters based on the above equations
for word in vocab:
    n_wi_spam = training_spam[word].sum()
    p_wi_spam = (n_wi_spam + alpha) / (n_spam + (alpha * n_vocabulary))
    p_wi_spam_dict[word] = p_wi_spam
    
    n_wi_ham = training_ham[word].sum()
    p_wi_ham = (n_wi_ham + alpha) / (n_ham + (alpha * n_vocabulary))
    p_wi_ham_dict[word] = p_wi_ham

### Creating the Spam Filter

The classifier can be understood as a function that:
1. Takes in as input a new message (w1, w2, ..., wn)
2. Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
3. Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:

    * If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    * If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    * If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

And for this, we're going to use the below equations:
\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}
\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

In [35]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Initiating values
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # 
    for word in message:
        if word in p_wi_spam_dict:
            p_spam_given_message *= p_wi_spam_dict[word]
        
        if word in p_wi_ham_dict:
            p_ham_given_message *= p_wi_ham_dict[word]
    
    # Displaying values
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [36]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3481290211300841e-25
P(Ham|message): 1.9368049028589875e-27
Label: Spam


In [37]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.4372375665888117e-25
P(Ham|message): 3.687530435009238e-21
Label: Ham


### Measuring our Filter Accuracy

Now, we're going to change our function a bit and use it to test our test dataset which has not been seen during the training process.

In [44]:
def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    # Initiating values
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # 
    for word in message:
        if word in p_wi_spam_dict:
            p_spam_given_message *= p_wi_spam_dict[word]
        
        if word in p_wi_ham_dict:
            p_ham_given_message *= p_wi_ham_dict[word]
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [49]:
# Values to calculate accuracy later
correct = 0
total = len(test_df)

# Classifying each SMS in the test dataset
# and saving the results to a separate column
test_df['predicted_label'] = test_df['SMS'].apply(classify)

# Measuring accuracy for each SMS
for row in test_df.iterrows():
    row = row[1]
    if row['Label'] == row['predicted_label']:
        correct += 1

accuracy = correct / total * 100
incorrect = total - correct

# Printing the results
print('Total:', total)
print('Correct:', correct)
print('Incorrect:', incorrect)
print('Accuracy: %' + str(round(accuracy, 2)))

Total: 1114
Correct: 1100
Incorrect: 14
Accuracy: %98.74


Our accuracy is %98.74 which is great. It only missed 14 SMS messages here.
# Thank you
[ha.hashem@outlook.com](ha.hashem@outlook.com)